In [2]:
# Hiển thị đường dẫn hiện tại
%pwd


'c:\\Users\\Admin\\Desktop\\2023.1\\BigData\\btl\\IT5384_Group5_Problem3\\DataProcessing'

In [1]:
!pip install pyspark


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
#config the connector jar file

In [8]:

spark = (SparkSession.builder.appName("SimpleSparkJob").master("spark://34.142.194.212:7077")
         .config("spark.jars", "C:\\GG_storage\\gcs-connector-latest-hadoop2.jar")
         .config("spark.executor.memory", "1G")  #excutor excute only 2G
        .config("spark.driver.memory","1G") 
        .config("spark.executor.cores","1") #Cluster use only 3 cores to excute as it has 3 server
        .config("spark.python.worker.memory","1G") # each worker use 1G to excute
        .config("spark.driver.maxResultSize","1G") #Maximum size of result is 3G
        .config("spark.kryoserializer.buffer.max","1024M")
         .getOrCreate())

#config the credential to identify the google cloud hadoop file 
spark.conf.set("google.cloud.auth.service.account.json.keyfile","C:\\GG_storage\\lucky-wall-393304-2a6a3df38253.json")
spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
spark._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.enable', 'true')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (231294422.py, line 1)

In [ ]:
spark

In [ ]:
tweets_path1 = "gs://it4043e-it5384/it5384/IT5384_Group5_Problem3/data_it5384_group5_problem3/data1.json"
tweets_path2 = "gs://it4043e-it5384/it5384/IT5384_Group5_Problem3/data_it5384_group5_problem3/data2.json"
# path to GG-cloud

In [ ]:
tweets_df1 = spark.read.option("multiline", "true").json(tweets_path1)
tweets_df2 = spark.read.option("multiline", "true").json(tweets_path1)


In [ ]:
back_up = tweets_df1

# Get the number of rows
num_rows = tweets_df1.count()

# Get the number of columns
num_columns = len(tweets_df1.columns)

# Print the shape of the DataFrame
print(f"Shape of the DataFrame: ({num_rows}, {num_columns})")


#get count columns null in data.json
for column in tweets_df1.columns:
    null_count = tweets_df1.filter(col(column).isNull()).count()
    print(f"NULL in {column}: {null_count}")

# Clean data

In [ ]:
replacement_value = 0  

columns_to_replace = ["Views", "Likes", "Reposts", "Replies"]

for column in columns_to_replace:
    tweets_df1 = tweets_df1.withColumn(column, when(tweets_df1[column].isNull(), replacement_value).otherwise(tweets_df1[column]))

replacement_value_user = "no"

for column in columns_to_replace:
    tweets_df1 = tweets_df1.withColumn(column, when(tweets_df1[column].isNull(), replacement_value_user).otherwise(tweets_df1[column]))

for column in tweets_df1.columns:
    null_count = tweets_df1.filter(col(column).isNull()).count()
    print(f"Null values in {column}: {null_count}")

#delete duplicate

df_no_duplicates = tweets_df1.dropDuplicates()


# Get the number of rows
num_rows = tweets_df1.count()

# Get the number of columns
num_columns = len(tweets_df1.columns)

# Print the shape of the DataFrame
print(f"Shape of the DataFrame: ({num_rows}, {num_columns})")



# storage

In [ ]:
output_directory = "gs://it4043e-it5384/it5384/IT5384_Group5_Problem3/data_it5384_group5_problem3"
tweets_df1_final: tweets_df1.DataFrame = tweets_df1.toPandas()
tweets_df1_final.to_json(output_directory + "data1.json", orient="records", lines=True)
tweets_df1_final.write.parquet("gs://it4043e-it5384/it5384/IT5384_Group5_Problem3/data_it5384_group5_problem3/data1.json", mode="overwrite")


In [ ]:
import json

def read_json_file(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except Exception as e:
        print(f"Error reading JSON file: {str(e)}")
        return None

# Đường dẫn tới file JSON
json_file_path = "path/to/your/file.json"

# Gọi hàm để đọc file JSON
json_data = read_json_file(json_file_path)

# Kiểm tra dữ liệu đã đọc
if json_data:
    print(json_data)
else:
    print("Failed to read JSON file.")
